# Filings Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import csv
from datetime import datetime

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

monthly total before running time.

In [ ]:
%%sql coop_gazette_incorporation  <<
select
  to_char(b.founding_date,'MON dd, yyyy')
||';'
|| CASE WHEN b.legal_type='CP' THEN 'CP '||substr(b.identifier,3,7)
        WHEN b.legal_type='XCP' THEN 'XCP'||substr(b.identifier,4,7) 
        ELSE 'BC '||substr(b.identifier,3,7)      
   END                 
||';'
||rpad(b.legal_name,150)
||rpad(' ',59)
from filings      f
    ,businesses   b    
where f.filing_type in ('incorporationApplication','amalgamationApplication') 
  and b.id=f.business_id 
  and f.source='LEAR' 
  and f.sent_to_gazette_date is null
order by b.founding_date;

In [ ]:
datestr = datetime.strftime(datetime.now(), '%Y%m%d')
coop_gazette_incorporation_filename = os.getenv('DATA_DIR', '')+'COOP_GAZETTE_INCORPORATION_' + datestr +'.TXT'

with open(coop_gazette_incorporation_filename, 'w') as f:
    if not coop_gazette_incorporation.DataFrame().empty:
        dfAsString = coop_gazette_incorporation.DataFrame().to_string(header=False, index=False)
        f.write(dfAsString)